In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
import yfinance as yf
from pandas_datareader import data as pdr
from datetime import datetime
from extract_target_columns import extract_target_columns

In [2]:
PATH_PETR4_PRICES_TWEETS = 'TimeSeries/PETR4_prices_and_tweets_NLI_scores.csv'

In [3]:
df_petr4 = pd.read_csv(PATH_PETR4_PRICES_TWEETS)

In [4]:
df_petr4['Date'] = pd.to_datetime(df_petr4['Date'])

In [5]:
def extract_target_columns(df):
    df_aux = df.copy()
    close_price_per_day_1_day_shifted = df_aux['Close'][1:]
    close_price_per_day_1_day_shifted = pd.concat([close_price_per_day_1_day_shifted, pd.Series(np.nan)], ignore_index=True)
    # Incluindo o valor do fechamento do dia seguinte no dataframe auxiliar
    df_aux['Close for next day'] = close_price_per_day_1_day_shifted
    # Removendo o útlimo dia de coleta de dados por não sabermos o preço de fechamento do dia seguinte
    df_aux = df_aux[:-1]

    df_aux['Close one day variation'] = df_aux['Close for next day'] - df_aux['Close']
    df_aux['hasRise'] = df_aux['Close one day variation'] > 0
    df_aux['hasRise'] = df_aux['hasRise'].map({True:1, False:0})

    return df_aux

In [6]:
df_petr4_with_targets = extract_target_columns(df_petr4)

In [8]:
df_petr4_with_targets.isna().sum()[df_petr4_with_targets.isna().sum() > 0]

Series([], dtype: int64)